In [1]:
import os
import pandas as pd
import numpy as np
from skimage.io import imread, imshow
from skimage import filters
from skimage import feature
from skimage.filters import prewitt_h,prewitt_v
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from csv import writer
import pickle
import re

In [2]:
def append_list_as_row(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [3]:
with open('df.pkl', 'rb') as file:  
    df = pickle.load(file)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,39992,39993,39994,39995,39996,39997,39998,39999,Names,overall_sentiment
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1.jpg,very_positive
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_10.png,positive
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_100.jpg,neutral
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1000.png,neutral
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1001.png,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6987,6987,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_995.jpeg,positive
6988,6988,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_996.jpg,negative
6989,6989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_997.jpg,very_positive
6990,6990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_998.jpg,positive


In [4]:
s={'very_positive':1,'positive':1, 'neutral':0,'negative':-1,'very_negative':-1}
df["overall_sentiment"]=df["overall_sentiment"].map(s)

In [23]:
needed = df.loc[df["overall_sentiment"] == -1]
needed

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,39992,39993,39994,39995,39996,39997,39998,39999,Names,overall_sentiment
14,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1010.png,-1
33,33,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1028.jpeg,-1
39,39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1033.jpg,-1
47,47,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1040.png,-1
53,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_1046.jpg,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,6969,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_979.png,-1
6974,6974,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_983.jpg,-1
6978,6978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_987.jpg,-1
6981,6981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,image_99.png,-1


In [24]:
needed = needed.drop(['Names','Unnamed: 0'], axis=1)
needed

,0,1,2,3,4,5,6,7,8,9,...,39991,39992,39993,39994,39995,39996,39997,39998,39999,overall_sentiment
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
33,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
39,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
6974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
6978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
6981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1


In [11]:
needed.to_csv('final_img.csv')

In [33]:
from skimage.transform import rotate
from skimage.io import imread, imshow
from skimage.transform import resize
for i in range(len(needed.index.values)):
    curr = np.array(needed.iloc[i][1:])
    curr = curr.reshape(200,200)
    n1 = rotate(curr, 40)
    x1 = np.array(list(np.concatenate(n1).flat))
    x1= x1.astype('bool')
    x1= x1.astype('int')
    x1 = np.append(x1, -1)
    x1 = np.insert(x1,0 , 99)
    append_list_as_row("final_img.csv",x1)

In [4]:
imp = df.drop(['Names', 'Unnamed: 0'], axis=1).copy(deep=True)
imp

,0,1,2,3,4,5,6,7,8,9,...,39991,39992,39993,39994,39995,39996,39997,39998,39999,overall_sentiment
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,very_positive
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neutral
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neutral
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive
6988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative
6989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,very_positive
6990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,positive


In [36]:
imp.to_csv("final_img.csv")